# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# DONE: Import the necessary libs
# For example: 
from datetime import datetime
import json
import os
from typing import Dict, List

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage
from lib.tooling import tool

import chromadb
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from tavily import TavilyClient

In [3]:
# DONE: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
# DONE: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# 
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

@tool
def retrieve_game(query: str) -> List:
    """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """

    print(f"Calling retrieve_game: query=\"{query}\"")

    results = collection.query(
        query_texts=[query],
        n_results=10
    )

    metadatas = results.get("metadatas", [[]])[0]

    result = [
        {
            "Platform": m.get("Platform"),
            "Name": m.get("Name"),
            "YearOfRelease": m.get("YearOfRelease"),
            "Description": m.get("Description")
        }
        for m in metadatas
    ]

    print(f"Returning: {json.dumps(result, indent=2)}")
    return result

#### Evaluate Retrieval Tool

In [5]:
# DONE: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

judge_llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

class EvaluationResult(BaseModel):
    useful: bool = Field(..., description="Whether the documents are useful to answer the question.")
    description: str = Field(..., description="Explanation of why the documents are or are not useful.")

class EvaluationResultList(BaseModel):
    evaluations: List[EvaluationResult]

@tool
def evaluate_retrieval(question: str, retrieved_docs: List) -> Dict:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 

    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database

    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """

    print(f"Calling evaluate_retrieval: question=\"{question}\", retrieved_docs={json.dumps(retrieved_docs, indent=2)}")

    response = judge_llm.invoke([
        SystemMessage(content="""
Your task is to evaluate if the documents are enough to respond the query.
Give a detailed explanation, so it's possible to take an action to accept it or not.
        """),
        UserMessage(content=f"""
The user asked:

### Question
{question}

### Retrieved Documents
{json.dumps(retrieved_docs, indent=2)}

Evaluate **each** document *individually*.

For each document:
1. Determine whether this single document is useful for answering the question.
2. Provide a short explanation.

Return the output in the following JSON structure:

{{
  "evaluations": [
    {{
      "useful": true/false,
      "description": "short explanation for doc #1"
    }},
    {{
      "useful": true/false,
      "description": "short explanation for doc #2"
    }},
    ...
  ]
}}
        """)
    ],
        response_format=EvaluationResultList
    )

    result = json.loads(response.content)

    print(f"Returning {json.dumps(result, indent=2)}")
    return result

#### Game Web Search Tool

In [6]:
# DONE: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 
@tool
def game_web_search(query: str) -> Dict:
    """
    Search the web using Tavily API
    args:
    - question: a question about game industry. 
    """

    print(f"Calling game_web_search: query=\"{query}\"")

    client = TavilyClient(api_key=TAVILY_API_KEY)

    # Perform the search
    search_result = client.search(
        query=query,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )

    # Format the results
    result = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }

    print(f"Returning {json.dumps(result, indent=2)}")
    return result

### Agent

In [7]:
# DONE: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

agent = Agent(
    model_name="gpt-4o-mini",
    instructions="""
You are UdaPlay, an AI Research Agent for the video game industry.

When answering questions about video games:
1. Find documents in the vector DB
2. Evaluate the quality of retrieved results
3. If no results are useful, use search the web for game information
4. Provide comprehensive answers, and include proper citations for web sources

You have access to these tools and should decide when to use each one based on the query and results.
Only use knowledge retrieved using tool calls.
    """,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    temperature=0.3
)

In [8]:
# DONE: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

In [9]:
def invoke_agent(question: str):
    run_object = agent.invoke(question)

    # Return last message generated by the agent
    last_message = run_object.get_final_state()["messages"][-1]
    print(f"Question: {question}")
    print(f"Answer  : {last_message.content}")

invoke_agent("When Pokémon Gold and Silver was released?")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Calling retrieve_game: query="Pokémon Gold and Silver release date"
Returning: [
  {
    "Platform": "Game Boy Color",
    "Name": "Pok\u00e9mon Gold and Silver",
    "YearOfRelease": 1999,
    "Description": "Second-generation Pok\u00e9mon games introducing new regions, Pok\u00e9mon, and gameplay mechanics."
  },
  {
    "Platform": "Game Boy Advance",
    "Name": "Pok\u00e9mon Ruby and Sapphire",
    "YearOfRelease": 2002,
    "Description": "Third-generation Pok\u00e9mon games set in the Hoenn region, featuring new Pok\u00e9mon and double battles."
  },
  {
    "Platform": "Nintendo Switch",
    "Name": "Mario Kart 8 Deluxe",
    "YearOfRelease": 2017,
    "Description": "An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics."
  },
  {
    "Platform": "Nintendo 64",
    "Name": "Super Mario 64",
    "YearOfReleas

In [10]:
invoke_agent("Which one was the first 3D platformer Mario game?")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Calling retrieve_game: query="first 3D platformer Mario game"
Returning: [
  {
    "Platform": "Nintendo 64",
    "Name": "Super Mario 64",
    "YearOfRelease": 1996,
    "Description": "A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to rescue Princess Peach."
  },
  {
    "Platform": "Super Nintendo Entertainment System (SNES)",
    "Name": "Super Mario World",
    "YearOfRelease": 1990,
    "Description": "A classic platformer where Mario embarks on a quest to save Princess Toadstool and Dinosaur Land from Bowser."
  },
  {
    "Platform": "Nintendo Switch",
    "Name": "Mario Kart 8 Deluxe",
    "YearOfRelease": 2017,
    "Description": "An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay mechanics."
  },
  {
    "Platform": "GameCube",
    "Name": "Super Smash Bros. Melee",


In [11]:
invoke_agent("Was Mortal Kombat X realeased for Playstation 5?")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
Calling retrieve_game: query="Mortal Kombat X release for Playstation 5"
Returning: [
  {
    "Platform": "PlayStation 5",
    "Name": "Marvel's Spider-Man 2",
    "YearOfRelease": 2023,
    "Description": "The sequel to the acclaimed Spider-Man game, featuring both Peter Parker and Miles Morales as playable characters."
  },
  {
    "Platform": "Xbox Series X|S",
    "Name": "Halo Infinite",
    "YearOfRelease": 2021,
    "Description": "The latest installment in the Halo franchise, featuring Master Chief's return in a new open-world setting."
  },
  {
    "Platform": "PlayStation 4",
    "Name": "Marvel's Spider-Man",
    "YearOfRelease": 2018,
    "Description": "An open-world superhero game that lets players swing through New York City as Spider-Man, battling iconic villains."
  },
  {
    "Platform": "PlayStation 3",
    "Name": "Gran Turismo 5",
    "YearOfR

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes